In [1]:
import numpy as np
import time

## Definition 4

In [2]:
def subword_subset(a,n):
    
    split_list = []
    len_a = len(a)
    for i in range(len_a):
        delim = len_a - i
        if delim < n:
            break
        split_list.append(a[i:i+n]) 
    return split_list

def Omega(arr):

#     t0 = time.time()
    
    subword_set = []
    for i in range(1, len(arr)):
        subword_set.extend(subword_subset(arr, i))
    subword_set.append(arr)
    
#     t1 = time.time()
#     print("Total time elapsed: ", t1-t0)
    
    return subword_set

for i in range(10):
    Omega('001010')

In [3]:
Omega1 = Omega('001010')
print(Omega1)
Omega2 = Omega('002020')

['0', '0', '1', '0', '1', '0', '00', '01', '10', '01', '10', '001', '010', '101', '010', '0010', '0101', '1010', '00101', '01010', '001010']


## Previous implementation of Defn 4 
##### Note the difference in efficiency 

In [4]:
def subword_len(a,n=1):
    
    split_list = [a[i:i+n] for i in range(0, len(a), 1)]
    
    return [x  for x in split_list if len(x) == n and len(x)>0]

def subword(a):
    #t0 = time.time()
    try:
        b=[a]
        nmax=len(a)
        for n in range(nmax):
            c=subword_len(a,n)
            if len(c)>0:
                b=np.append(b,c)
        return b

    except: 
        return b

    #t1 = time.time()
    #print("Total time elapsed: ", t1-t0)

subword('34')
# for i in range(10):
#     print(subword('001010'))

array(['34', '3', '4'], dtype='|S2')

## Definition 5 - Jacard Similarity

##### Same as prior implementation as per ishanuc

In [5]:
def get_unweightedJac(S1,S2):
    return (len(set(S1).union(S2))-len(set(S1).intersection(S2)))/(0.0+len(set(S1).union(S2)))

# Definition 6 - Weighted Jacard Sequential Distance

In [6]:
def sum_weight_list(a):
    t0 = time.time()
    for i in range(len(a)):
        a[i]
    t1 = time.time()
   #print("list time: ", t1 - t0)
    return t1 - t0

def sum_weight_list_alt(a):
    t0 = time.time()
    for fish in a:
        fish
    t1 = time.time()
    #print("list alt time: ", t1 - t0)
    return t1 - t0
    
def sum_weight_set(a):
    t0 = time.time()
    for fish in a:
        fish
    t1 = time.time()
    #print("set time: ", t1 - t0)
    return t1 - t0

listtime = 0
listalttime = 0
settime = 0
for i in range(100):
    listtime += sum_weight_list(list(set(Omega1).union(Omega2)))
    listalttime += sum_weight_list_alt(list(set(Omega1).union(Omega2)))
    settime += sum_weight_set(set(Omega1).union(Omega2))

print("100 runs list: ", listtime)
print("100 runs list alt: ", listalttime)
print("100 runs set: ", settime)

('100 runs list: ', 0.00031638145446777344)
('100 runs list alt: ', 0.0001671314239501953)
('100 runs set: ', 0.00020956993103027344)


In [7]:
def get_weightedJac(S1,S2, sum_weight):
    sum_weight_union = sum_weight(list(set(S1).union(S2)))
    return (sum_weight_union - sum_weight(list(set(S1).intersection(S2))))/(0.0+sum_weight_union)

# Generate rando data

In [8]:
p0 = np.asarray([0.1, 0.05, 0.05, 0.2, 0.4, 0.2])
dummy = np.random.choice(np.arange(0, 6), size = (3,3),p = p0)
dummy

array([[0, 4, 0],
       [2, 0, 4],
       [1, 3, 2]])

In [9]:
print(dummy[:,0].shape)

(3,)


In [10]:
# make sure to weight each sequence by population prevalence 
# (basically use the probability vector that we will be shifting around)
# Implement the weighted Jacard Later

def weight(p0_vector, i):
    return 1/p0_vector[i]

def jacMatrix(vec):
    length = vec.shape[0]
    jac_mat = np.zeros(shape = (length, length))
    print(vec[1])
    for i in range(length):
        for j in range(length):
            jac_mat[i][j] = get_unweightedJac(subword(vec[i]), subword(vec[j]))
    return jac_mat
                       
    
    
    
test_jac = jacMatrix(dummy)
test_jac        

[2 0 4]


array([[0.        , 0.33333333, 1.        ],
       [0.33333333, 0.        , 0.8       ],
       [1.        , 0.8       , 0.        ]])

# Quasi Metric

In [11]:
# reverse dictionary that assigns probability from original probability vector 
# to an index for use in next fn
def prob_reverse(prob_vec):
    rev_probs = {}
    for i in range(prob_vec.shape[0]):
        rev_probs[i] = prob_vec[i]
    return rev_probs




rev_dict = prob_reverse(p0)
rev_dict

{0: 0.1, 1: 0.05, 2: 0.05, 3: 0.2, 4: 0.4, 5: 0.2}

In [12]:
# reverse dictionary that assigns probability to indexed history
# we initialize this then we, in the future only edit the hist_prob_dict for future histories
def hist_probs(histories, prob_dict):
    hist_prob = {}
    for i in range(histories.shape[0]):
        hist_prob[i] = prob_dict[histories[i][0]]
    return hist_prob




hist_prob = hist_probs(dummy, rev_dict)
hist_prob

{0: 0.1, 1: 0.05, 2: 0.05}

In [13]:
# quasi metric matrix normalization constant
def qm_matrix_normalization(hist_probs, i, j):
    return .5 * (((1/hist_probs[i]) + (1/hist_probs[j])) - 2)




test = qm_matrix_normalization(hist_prob, 1, 1)
test

19.0

In [14]:
# takes in jacard matrix and probabilty vector
# returns quasi metric matrix
# remember we have not used the weight fn, so this needs to be later implemented
# can combine later with jac_matrix maker so we dont have to do 2 runs of matrix
def qm_Matrix(vec, jac_mat, prob_dict):
    length = jac_mat.shape[0]
    qm_mat = np.zeros(shape = (length, length))
    for i in range(length):
        for j in range(length):
            qm_mat[i][j] = qm_matrix_normalization(prob_dict, i, j) * jac_mat[i][j]
    return qm_mat




test_qm = qm_Matrix(dummy, test_jac, hist_prob)
test_qm

array([[ 0.        ,  4.66666667, 14.        ],
       [ 4.66666667,  0.        , 15.2       ],
       [14.        , 15.2       ,  0.        ]])

# Epsilon Neighborhood Step

In [67]:
# given an epsilon, figures out which indexes are in the epsilon neighborhood of the matrix
# basically for each row, we see which things in epsilon neighborhood then keep indexes,
# which we then use to figure out probabilities of occurrence
# qm matrix is gonna be symmetric, so we do calculations per column
def epsilon_indexes(qm_matrix, epsilon):
    col_indexes = {}
    index_list = []
    for i in range(qm_matrix.shape[0]):
        for j in range(qm_matrix.shape[0]):
            if qm_matrix[i][j] < epsilon:
                index_list.append(j)
        col_indexes[i] = index_list
        index_list = []
    return col_indexes

test = epsilon_indexes(test_qm, 10)
test

{0: [0, 1], 1: [0, 1], 2: [2]}

In [110]:
# updates probability of given history
# will cache rpt probabilities later
def hist_prob_update(histories, epsilon_indexes, prob_vectors, idx):
    pvecs = {}
    for key in epsilon_indexes:
        yesval = histories[key][idx]
        yes = 0 
        no = 0
        for i in range(len(epsilon_indexes[key])):
            histval = histories[key][epsilon_indexes[key][i]]
            if yesval == histval:
                yes = yes + 1
            else:
                no = no + 1
        #print(yes,no)
        prob = yes/(yes * 1.0 + no)
 
        prob_vecs[key] = prob_vectors[key] * prob
    return prob_vectors
        
#hist_prob_update(dummy, test, hist_prob, 1)

In [111]:
# is quasi metric, have to do bigger test : test_dummy = np.asarray([[0,1,2],[1,0,4],[2,4,0]])
epsilon = 3

edxs = epsilon_indexes(test_dummy, 3)
prob_vecs = {0:2/3.0,1:1/3.0, 2:1/3.0}
pvecs = hist_prob_update(test_dummy, edxs, prob_vecs, 1)
pvecs

0 1
1 1
2 1
0.333333333333
yes reset
1 0
0 0
0.5
yes reset
2 4
0 4
0.0
yes reset


{0: 0.2222222222222222, 1: 0.16666666666666666, 2: 0.0}